# Example: Graph generation from a gene

This notebook will show you how to use the tool to generate a KG on the underlying data.

In [1]:
new_path = "E:\BioDataFuse\pyBiodatafuse"

import os

os.chdir(new_path)

# Set the current working directory
current_dir = os.getcwd()
print("Current directory:", current_dir)

Current directory: E:\BioDataFuse\pyBiodatafuse


In [2]:
# Import modules
import pickle

import matplotlib.pyplot as plt
import networkx as nx
import pandas as pd
from dotenv import load_dotenv

from pyBiodatafuse import id_mapper
from pyBiodatafuse.annotators import (
    bgee,
    disgenet,
    minerva,
    molmedb,
    opentargets,
    pubchem,
    stringdb,
    wikipathways,
)
from pyBiodatafuse.constants import (
    BGEE,
    DISGENET,
    MINERVA,
    MOLMEDB_INHIBITED_COL,
    MOLMEDB_INHIBITOR_COL,
    OPENTARGETS_COMPOUND_COL,
    OPENTARGETS_DISEASE_COL,
    OPENTARGETS_GO_COL,
    OPENTARGETS_REACTOME_COL,
    PUBCHEM_ASSAYS_COL,
    STRING,
    WIKIPATHWAYS,
)
from pyBiodatafuse.graph import generator
from pyBiodatafuse.utils import combine_sources

### Load the input list and convert it to a dataframe

In [3]:
# genes_of_interest = """AAGRN
# ALG14
# ALG2
# CHAT
# CHD8
# CHRNA1
# CHRNB1
# CHRND
# CHRNE
# CHRNG
# COL13A1
# COLQ
# DOK7
# DPAGT1
# GFPT1
# GMPPB
# LAMA5
# LAMB2
# LRP4
# MUSK
# MYO9A
# PLEC
# PREPL
# PURA
# RAPSN
# RPH3A
# SCN4A
# SLC18A3
# SLC25A1
# SLC5A7
# SNAP25
# SYT2
# TOR1AIP1
# UNC13A
# VAMP1"""
genes_of_interest = """CHRNG
DMD
AHR
SCN4A
SLC25A1
HTR3A"""
# genes_of_interest = "DMD"
gene_list = genes_of_interest.split("\n")
len(gene_list)

6

In [4]:
data_input = pd.DataFrame(gene_list, columns=["identifier"])
data_input.head()

,identifier
0,CHRNG
1,DMD
2,AHR
3,SCN4A
4,SLC25A1


### Entity resolution using BridgeDB

In [5]:
bridgdb_df, bridgdb_metadata = id_mapper.bridgedb_xref(
    identifiers=data_input,
    input_species="Human",
    input_datasource="HGNC",
    output_datasource="All",
)
bridgdb_df.head()

,identifier,identifier.source,target,target.source
0,CHRNG,HGNC,GO:0042391,Gene Ontology
1,CHRNG,HGNC,A_23_P5718,Agilent
2,CHRNG,HGNC,GO:0016021,Gene Ontology
3,CHRNG,HGNC,GO:0016020,Gene Ontology
4,CHRNG,HGNC,GO:0006936,Gene Ontology


### Gene expression from Bgee

In [6]:
bgee_df, bgee_metadata = bgee.get_gene_expression(bridgedb_df=bridgdb_df)
bgee_df.head()

,identifier,identifier.source,target,target.source,Bgee
0,AHR,HGNC,ENSG00000106546,Ensembl,"[{'anatomical_entity_id': 'UBERON_0000178', 'a..."
1,CHRNG,HGNC,ENSG00000196811,Ensembl,"[{'anatomical_entity_id': 'UBERON_0000178', 'a..."
2,DMD,HGNC,ENSG00000198947,Ensembl,"[{'anatomical_entity_id': 'UBERON_0000178', 'a..."
3,HTR3A,HGNC,ENSG00000166736,Ensembl,"[{'anatomical_entity_id': 'UBERON_0000178', 'a..."
4,SCN4A,HGNC,ENSG00000007314,Ensembl,"[{'anatomical_entity_id': 'UBERON_0000178', 'a..."


In [7]:
bgee_df[BGEE][0]

[{'anatomical_entity_id': 'UBERON_0000178',
  'anatomical_entity_name': 'blood',
  'expression_level': 80.23869999999998,
  'confidence_level_id': 'CIO_0000029',
  'confidence_level_name': 'high confidence level',
  'developmental_stage_id': 'UBERON_0000104',
  'developmental_stage_name': 'life cycle'},
 {'anatomical_entity_id': 'UBERON_0002371',
  'anatomical_entity_name': 'bone marrow',
  'expression_level': 77.48559,
  'confidence_level_id': 'CIO_0000029',
  'confidence_level_name': 'high confidence level',
  'developmental_stage_id': 'UBERON_0000104',
  'developmental_stage_name': 'life cycle'},
 {'anatomical_entity_id': 'UBERON_0000955',
  'anatomical_entity_name': 'brain',
  'expression_level': 64.3383,
  'confidence_level_id': 'CIO_0000029',
  'confidence_level_name': 'high confidence level',
  'developmental_stage_id': 'UBERON_0000104',
  'developmental_stage_name': 'life cycle'},
 {'anatomical_entity_id': 'UBERON_0000310',
  'anatomical_entity_name': 'breast',
  'expression_le

### Disease annotatation from DisGeNet


ADD your DISGENET API KEY in the main folder

**1)** Create a .env File and add DISGENET_API_KEY to it:

DISGENET_API_KEY="your-API-key-value"

**2)** Install *python-dotenv*:
```
pip install python-dotenv
```

In [6]:
# Read the .env File
load_dotenv()
# Retrieve the key from the environment variable
disgenet_api_key = os.getenv("DISGENET_API_KEY")

In [7]:
disgenet_df, disgenet_metadata = disgenet.get_gene_disease(
    api_key=disgenet_api_key, bridgedb_df=bridgdb_df
)
disgenet_df.head()

e:\BioDataFuse\pyBiodatafuse\.venv\lib\site-packages\urllib3\connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.disgenet.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
e:\BioDataFuse\pyBiodatafuse\.venv\lib\site-packages\urllib3\connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.disgenet.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
e:\BioDataFuse\pyBiodatafuse\.venv\lib\site-packages\urllib3\connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.disgenet.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
e:\BioDataFuse\pyBiodataf

,identifier,identifier.source,target,target.source,DISGENET
0,AHR,HGNC,196,NCBI Gene,"[{'HPO': 'HPO_HP:0100013', 'NCI': 'NCI_C2910',..."
1,CHRNG,HGNC,1146,NCBI Gene,"[{'HPO': '', 'NCI': 'NCI_C101039', 'OMIM': 'OM..."
2,DMD,HGNC,1756,NCBI Gene,"[{'HPO': '', 'NCI': 'NCI_C75482', 'OMIM': 'OMI..."
3,HTR3A,HGNC,3359,NCBI Gene,"[{'HPO': 'HPO_HP:0100753', 'NCI': 'NCI_C3362',..."
4,SCN4A,HGNC,6329,NCBI Gene,"[{'HPO': '', 'NCI': 'NCI_C122788', 'OMIM': 'OM..."


In [8]:
disgenet_df[DISGENET][0]

[{'HPO': 'HPO_HP:0100013',
  'NCI': 'NCI_C2910',
  'OMIM': '',
  'MONDO': 'MONDO_0021100',
  'ORDO': '',
  'ICD10': '',
  'EFO': 'EFO_0003869',
  'DO': 'DO_3459, DO_1612',
  'MESH': 'MESH_D001943',
  'UMLS': 'UMLS_C1458155',
  'ICD9CM': '',
  'disease_name': 'Mammary Neoplasms',
  'disease_type': 'disease',
  'disease_umlscui': 'C1458155',
  'score': 0.8500000000000001,
  'ei': 0.9176470588235294,
  'el': None},
 {'HPO': 'HPO_HP:0001370',
  'NCI': 'NCI_C2884',
  'OMIM': 'OMIM_607218, OMIM_180300',
  'MONDO': 'MONDO_0008383',
  'ORDO': '',
  'ICD10': 'ICD10_M06.9',
  'EFO': 'EFO_0000685',
  'DO': 'DO_7148',
  'MESH': 'MESH_D001172',
  'UMLS': 'UMLS_C0003873',
  'ICD9CM': 'ICD9CM_714.0',
  'disease_name': 'Rheumatoid Arthritis',
  'disease_type': 'disease',
  'disease_umlscui': 'C0003873',
  'score': 0.8,
  'ei': 0.9523809523809523,
  'el': None},
 {'HPO': 'HPO_HP:0001513',
  'NCI': 'NCI_C159658, NCI_C3283',
  'OMIM': 'OMIM_601665',
  'MONDO': 'MONDO_0011122, MONDO_0019182',
  'ORDO': ''

### Disease annotation from OpenTargets

In [9]:
disease_df, opentargets_disease_metadata = opentargets.get_gene_disease_associations(
    bridgedb_df=bridgdb_df
)
disease_df.head()

,identifier,identifier.source,target,target.source,OpenTargets_Diseases
0,AHR,HGNC,ENSG00000106546,Ensembl,"[{'disease_id': 'umls:C0033860', 'disease_name..."
1,CHRNG,HGNC,ENSG00000196811,Ensembl,"[{'disease_id': 'umls:C0085631', 'disease_name..."
2,DMD,HGNC,ENSG00000198947,Ensembl,"[{'disease_id': 'umls:C0013264', 'disease_name..."
3,HTR3A,HGNC,ENSG00000166736,Ensembl,"[{'disease_id': 'EFO_0003843', 'disease_name':..."
4,SCN4A,HGNC,ENSG00000007314,Ensembl,"[{'disease_id': 'EFO_0000432', 'disease_name':..."


In [10]:
disease_df[OPENTARGETS_DISEASE_COL][0]

[{'disease_id': 'umls:C0033860',
  'disease_name': 'psoriasis',
  'therapeutic_areas': 'OTAR_0000018:genetic, familial or congenital disease, EFO_0000540:immune system disease, EFO_0010285:integumentary system disease'},
 {'disease_id': 'EFO_0000274',
  'disease_name': 'atopic eczema',
  'therapeutic_areas': 'OTAR_0000018:genetic, familial or congenital disease, EFO_0000540:immune system disease, EFO_0010285:integumentary system disease'},
 {'disease_id': 'MONDO_0011292',
  'disease_name': 'dermatitis, atopic',
  'therapeutic_areas': 'OTAR_0000018:genetic, familial or congenital disease, EFO_0000540:immune system disease, EFO_0010285:integumentary system disease'}]

### Pathways from MINERVA

In [12]:
minerva_df, minerva_metadata = minerva.get_gene_minerva_pathways(
    bridgdb_df, map_name="COVID19 Disease Map"
)
minerva_df.head()

    identifier identifier.source           target target.source
23       CHRNG              HGNC  ENSG00000196811       Ensembl
502        DMD              HGNC  ENSG00000198947       Ensembl
675        AHR              HGNC  ENSG00000106546       Ensembl
769      SCN4A              HGNC  ENSG00000007314       Ensembl
787    SLC25A1              HGNC  ENSG00000100075       Ensembl
890      HTR3A              HGNC  ENSG00000166736       Ensembl
     symbol                 pathway_label  pathway_gene_count  pathway_id  \
2643    AHR  Kynurenine synthesis pathway                  45         953   

                                                   refs           target  \
2643  [{'link': 'https://www.genenames.org/data/gene...  ENSG00000106546   

         type  
2643  Protein  


,identifier,identifier.source,target,target.source,MINERVA
0,AHR,HGNC,ENSG00000106546,Ensembl,"[{'pathway_id': 953.0, 'pathway_label': 'Kynur..."
1,CHRNG,HGNC,ENSG00000196811,Ensembl,"[{'pathway_id': nan, 'pathway_label': nan, 'pa..."
2,DMD,HGNC,ENSG00000198947,Ensembl,"[{'pathway_id': nan, 'pathway_label': nan, 'pa..."
3,HTR3A,HGNC,ENSG00000166736,Ensembl,"[{'pathway_id': nan, 'pathway_label': nan, 'pa..."
4,SCN4A,HGNC,ENSG00000007314,Ensembl,"[{'pathway_id': nan, 'pathway_label': nan, 'pa..."


In [12]:
minerva_df[MINERVA][0]

[{'pathway_id': 953.0,
  'pathway_label': 'Kynurenine synthesis pathway',
  'pathway_gene_count': 45.0}]

### Pathways from WikiPathways

In [13]:
wikipathways_df, wikipathways_metadata = wikipathways.get_gene_wikipathways(bridgedb_df=bridgdb_df)
wikipathways_df.head()

,identifier,identifier.source,target,target.source,WikiPathways
0,AHR,HGNC,196,NCBI Gene,"[{'pathway_id': 'WP5044', 'pathway_label': 'Ky..."
1,CHRNG,HGNC,1146,NCBI Gene,"[{'pathway_id': nan, 'pathway_label': nan, 'pa..."
2,DMD,HGNC,1756,NCBI Gene,"[{'pathway_id': 'WP2858', 'pathway_label': 'Ec..."
3,HTR3A,HGNC,3359,NCBI Gene,"[{'pathway_id': 'WP706', 'pathway_label': 'Sud..."
4,SCN4A,HGNC,6329,NCBI Gene,"[{'pathway_id': nan, 'pathway_label': nan, 'pa..."


In [14]:
wikipathways_df[WIKIPATHWAYS][0]

[{'pathway_id': 'WP5044',
  'pathway_label': 'Kynurenine pathway and links to cell senescence',
  'pathway_gene_count': 10.0},
 {'pathway_id': 'WP3893',
  'pathway_label': 'Development and heterogeneity of the ILC family',
  'pathway_gene_count': 32.0},
 {'pathway_id': 'WP3869',
  'pathway_label': 'Cannabinoid receptor signaling',
  'pathway_gene_count': 31.0},
 {'pathway_id': 'WP4673',
  'pathway_label': 'Male infertility',
  'pathway_gene_count': 145.0},
 {'pathway_id': 'WP2586',
  'pathway_label': 'Aryl hydrocarbon receptor pathway',
  'pathway_gene_count': 45.0},
 {'pathway_id': 'WP236',
  'pathway_label': 'Adipogenesis',
  'pathway_gene_count': 131.0},
 {'pathway_id': 'WP2873',
  'pathway_label': 'Aryl hydrocarbon receptor pathway',
  'pathway_gene_count': 46.0},
 {'pathway_id': 'WP5088',
  'pathway_label': 'Prostaglandin signaling',
  'pathway_gene_count': 31.0},
 {'pathway_id': 'WP5115',
  'pathway_label': 'Network map of SARS-CoV-2 signaling',
  'pathway_gene_count': 251.0},
 {

### Reactome pathways from OpenTargets

In [15]:
reactome_process_df, opentargets_process_metadata = opentargets.get_gene_reactome_pathways(
    bridgedb_df=bridgdb_df
)
reactome_process_df.head()

,identifier,identifier.source,target,target.source,OpenTargets_Reactome
0,AHR,HGNC,ENSG00000106546,Ensembl,"[{'pathway_label': 'Endogenous sterols', 'path..."
1,CHRNG,HGNC,ENSG00000196811,Ensembl,[{'pathway_label': 'Highly sodium permeable po...
2,DMD,HGNC,ENSG00000198947,Ensembl,[{'pathway_label': 'Striated Muscle Contractio...
3,HTR3A,HGNC,ENSG00000166736,Ensembl,[{'pathway_label': 'Neurotransmitter receptors...
4,SCN4A,HGNC,ENSG00000007314,Ensembl,[{'pathway_label': 'Phase 0 - rapid depolarisa...


In [16]:
reactome_process_df[OPENTARGETS_REACTOME_COL][0]

[{'pathway_label': 'Endogenous sterols', 'pathway_id': 'R-HSA-211976'},
 {'pathway_label': 'PPARA activates gene expression',
  'pathway_id': 'R-HSA-1989781'},
 {'pathway_label': 'Phase I - Functionalization of compounds',
  'pathway_id': 'R-HSA-211945'},
 {'pathway_label': 'Xenobiotics', 'pathway_id': 'R-HSA-211981'},
 {'pathway_label': 'Aryl hydrocarbon receptor signalling',
  'pathway_id': 'R-HSA-8937144'}]

### Gene Ontology from OpenTargets

In [17]:
go_process_df, opentargets_go_metadata = opentargets.get_gene_go_process(bridgedb_df=bridgdb_df)
go_process_df.head()

,identifier,identifier.source,target,target.source,OpenTargets_GO
0,AHR,HGNC,ENSG00000106546,Ensembl,"[{'go_id': 'GO:0005667', 'go_name': 'transcrip..."
1,CHRNG,HGNC,ENSG00000196811,Ensembl,"[{'go_id': 'GO:0015464', 'go_name': 'acetylcho..."
2,DMD,HGNC,ENSG00000198947,Ensembl,"[{'go_id': 'GO:0016010', 'go_name': 'dystrophi..."
3,HTR3A,HGNC,ENSG00000166736,Ensembl,"[{'go_id': 'GO:1904602', 'go_name': 'serotonin..."
4,SCN4A,HGNC,ENSG00000007314,Ensembl,"[{'go_id': 'GO:0035725', 'go_name': 'sodium io..."


In [18]:
go_process_df[OPENTARGETS_GO_COL][0]

[{'go_id': 'GO:0005667',
  'go_name': 'transcription regulator complex',
  'go_type': 'C'},
 {'go_id': 'GO:0004879',
  'go_name': 'nuclear receptor activity',
  'go_type': 'F'},
 {'go_id': 'GO:0005634', 'go_name': 'nucleus', 'go_type': 'C'},
 {'go_id': 'GO:0046982',
  'go_name': 'protein heterodimerization activity',
  'go_type': 'F'},
 {'go_id': 'GO:0009410',
  'go_name': 'response to xenobiotic stimulus',
  'go_type': 'P'},
 {'go_id': 'GO:0000976',
  'go_name': 'transcription cis-regulatory region binding',
  'go_type': 'F'},
 {'go_id': 'GO:0005829', 'go_name': 'cytosol', 'go_type': 'C'},
 {'go_id': 'GO:0030888',
  'go_name': 'regulation of B cell proliferation',
  'go_type': 'P'},
 {'go_id': 'GO:0009636',
  'go_name': 'response to toxic substance',
  'go_type': 'P'},
 {'go_id': 'GO:0051879', 'go_name': 'Hsp90 protein binding', 'go_type': 'F'},
 {'go_id': 'GO:0001094',
  'go_name': 'TFIID-class transcription factor complex binding',
  'go_type': 'F'},
 {'go_id': 'GO:0001568',
  'go_n

### Gene location annotation from OpenTargets

In [19]:
loc_df, opentargets_loc_metadata = opentargets.get_gene_location(bridgedb_df=bridgdb_df)
loc_df.head()

,identifier,identifier.source,target,target.source,OpenTargets_Location
0,AHR,HGNC,ENSG00000106546,Ensembl,"[{'location_id': 'SL-0086', 'location': 'Cytop..."
1,CHRNG,HGNC,ENSG00000196811,Ensembl,"[{'location_id': 'SL-0219', 'location': 'Posts..."
2,DMD,HGNC,ENSG00000198947,Ensembl,"[{'location_id': 'SL-0039', 'location': 'Cell ..."
3,HTR3A,HGNC,ENSG00000166736,Ensembl,"[{'location_id': 'SL-0219', 'location': 'Posts..."
4,SCN4A,HGNC,ENSG00000007314,Ensembl,"[{'location_id': 'SL-0039', 'location': 'Cell ..."


In [20]:
loc_df[OPENTARGETS_LOCATION_COL][0]

[{'location_id': 'SL-0086',
  'location': 'Cytoplasm',
  'subcellular_location': 'Cellular component'},
 {'location_id': 'SL-0191',
  'location': 'Nucleus',
  'subcellular_location': 'Cellular component'},
 {'location_id': 'SL-0091',
  'location': 'Cytosol',
  'subcellular_location': 'Cytosol'},
 {'location_id': 'SL-0190',
  'location': 'Nucleoplasm',
  'subcellular_location': 'Nucleoplasm'}]

### Compounds from OpenTarget

In [21]:
drug_df, opentargets_drug_metadata = opentargets.get_gene_compound_interactions(
    bridgedb_df=bridgdb_df
)
drug_df.head()

E:\BioDataFuse\pyBiodatafuse\src\pyBiodatafuse\annotators\opentargets.py:593: UserWarning: Not all values in column 'drugbank_id' have the correct type 'typing.Union[str, NoneType, float]'.
  check_columns_against_constants(
E:\BioDataFuse\pyBiodatafuse\src\pyBiodatafuse\annotators\opentargets.py:593: UserWarning: Not all values in column 'compound_cid' have the correct type 'typing.Union[str, NoneType, float]'.
  check_columns_against_constants(
E:\BioDataFuse\pyBiodatafuse\src\pyBiodatafuse\annotators\opentargets.py:593: UserWarning: Not all values in column 'adverse_effect_count' have the correct type 'typing.Union[int, NoneType, float]'.
  check_columns_against_constants(
E:\BioDataFuse\pyBiodatafuse\src\pyBiodatafuse\annotators\opentargets.py:593: UserWarning: Not all values in column 'adverse_effect' have the correct type 'typing.Union[str, NoneType, float]'.
  check_columns_against_constants(


,identifier,identifier.source,target,target.source,OpenTargets_Compounds
0,AHR,HGNC,ENSG00000106546,Ensembl,"[{'chembl_id': 'CHEMBL259571', 'drugbank_id': ..."
1,CHRNG,HGNC,ENSG00000196811,Ensembl,"[{'chembl_id': 'CHEMBL1200641', 'drugbank_id':..."
2,DMD,HGNC,ENSG00000198947,Ensembl,"[{'chembl_id': 'CHEMBL2108278', 'drugbank_id':..."
3,HTR3A,HGNC,ENSG00000166736,Ensembl,"[{'chembl_id': 'CHEMBL56564', 'drugbank_id': '..."
4,SCN4A,HGNC,ENSG00000007314,Ensembl,"[{'chembl_id': 'CHEMBL1077896', 'drugbank_id':..."


In [22]:
drug_df[OPENTARGETS_COMPOUND_COL][0]

[{'chembl_id': 'CHEMBL259571',
  'drugbank_id': 'DB06083',
  'compound_cid': None,
  'compound_name': 'TAPINAROF',
  'is_approved': True,
  'relation': 'activates',
  'adverse_effect_count': nan,
  'adverse_effect': None}]

### Transporter inhibitors from MolMeDB

In [6]:
inhibitor_df, inhibitor_metadata = molmedb.get_gene_compound_inhibitor(bridgedb_df=bridgdb_df)
inhibitor_df.head()

,identifier,identifier.source,target,target.source,MolMeDB_transporter_inhibitor
0,AHR,HGNC,A0A024R9Z8,Uniprot-TrEMBL,"[{'compound_name': nan, 'InChIKey': nan, 'SMIL..."
1,CHRNG,HGNC,A0A6F7YAP6,Uniprot-TrEMBL,"[{'compound_name': nan, 'InChIKey': nan, 'SMIL..."
2,DMD,HGNC,A0A075B6G3,Uniprot-TrEMBL,"[{'compound_name': nan, 'InChIKey': nan, 'SMIL..."
3,HTR3A,HGNC,A0A0B4J205,Uniprot-TrEMBL,"[{'compound_name': nan, 'InChIKey': nan, 'SMIL..."
4,SCN4A,HGNC,P35499,Uniprot-TrEMBL,"[{'compound_name': '3-phenyl-1h-pyrazole', 'In..."


In [7]:
inhibitor_metadata

{'datasource': 'MolMeDB',
 'query': {'size': 50,
  'number_of_added_nodes': 247,
  'number_of_added_edges': 247,
  'input_type': 'Uniprot-TrEMBL',
  'time': '0:00:00.876801',
  'date': '2024-08-01 15:47:20',
  'url': 'https://idsm.elixir-czech.cz/sparql/endpoint/molmedb'}}

In [8]:
inhibitor_df[MOLMEDB_INHIBITOR_COL][56]

[{'compound_name': nan,
  'InChIKey': nan,
  'SMILES': nan,
  'compound_cid': nan,
  'molmedb_id': nan,
  'source_doi': nan,
  'source_pmid': nan,
  'chebi_id': nan,
  'drugbank_id': nan}]

### Screening results of compounds on proteins encoded by genes annotation by PubChem

In [25]:
pubchem_assay_df, pubchem_assay_metadata = pubchem.get_protein_molecule_screened(
    bridgedb_df=bridgdb_df
)
pubchem_assay_df.head()

,identifier,identifier.source,target,target.source,PubChem_Assays
0,AHR,HGNC,A0A024R9Z8,Uniprot-TrEMBL,"[{'pubchem_assay_id': nan, 'assay_type': nan, ..."
1,AHR,HGNC,A0A2R8Y7G1,Uniprot-TrEMBL,"[{'pubchem_assay_id': nan, 'assay_type': nan, ..."
2,AHR,HGNC,P35869,Uniprot-TrEMBL,"[{'pubchem_assay_id': nan, 'assay_type': nan, ..."
3,CHRNG,HGNC,A0A6F7YAP6,Uniprot-TrEMBL,"[{'pubchem_assay_id': nan, 'assay_type': nan, ..."
4,CHRNG,HGNC,P07510,Uniprot-TrEMBL,"[{'pubchem_assay_id': nan, 'assay_type': nan, ..."


In [26]:
pubchem_assay_df[PUBCHEM_ASSAYS_COL][10]

[{'pubchem_assay_id': nan,
  'assay_type': nan,
  'outcome': nan,
  'compound_cid': nan,
  'compound_name': nan,
  'SMILES': nan,
  'InChI': nan}]

### Protein-Protein interactions from STRING

In [27]:
ppi_df, ppi_metadata = stringdb.get_ppi(bridgedb_df=bridgdb_df)
ppi_df.head()

,identifier,identifier.source,target,target.source,StringDB
0,CHRNG,HGNC,ENSG00000196811,Ensembl,"[{'stringdb_link_to': 'SCN4A', 'Ensembl': 'ENS..."
1,DMD,HGNC,ENSG00000198947,Ensembl,"[{'stringdb_link_to': 'SCN4A', 'Ensembl': 'ENS..."
2,AHR,HGNC,ENSG00000106546,Ensembl,[]
3,SCN4A,HGNC,ENSG00000007314,Ensembl,"[{'stringdb_link_to': 'DMD', 'Ensembl': 'ENSP0..."
4,SLC25A1,HGNC,ENSG00000100075,Ensembl,[]


In [28]:
ppi_df[STRING][0]

[{'stringdb_link_to': 'SCN4A', 'Ensembl': 'ENSP00000396320', 'score': 0.454}]

### Combing all the results into single dataframe

In [30]:
combined_df = combine_sources(
    [
        bgee_df,
        minerva_df,
        wikipathways_df,
        reactome_process_df,
        go_process_df,
        loc_df,
        disgenet_df,
        disease_df,
        drug_df,
        inhibitor_df,
        pubchem_assay_df,
        ppi_df,
    ]
)

In [31]:
combined_df.head(4)

,identifier,identifier.source,target,target.source,Bgee,MINERVA,WikiPathways,OpenTargets_Reactome,OpenTargets_GO,OpenTargets_Location,OpenTargets_Diseases,OpenTargets_Compounds,MolMeDB_transporter_inhibitor,PubChem_Assays,StringDB
0,AHR,HGNC,ENSG00000106546,Ensembl,"[{'anatomical_entity_id': 'UBERON_0000178', 'a...","[{'pathway_id': 953.0, 'pathway_label': 'Kynur...","[{'pathway_id': 'WP5044', 'pathway_label': 'Ky...","[{'pathway_label': 'Endogenous sterols', 'path...","[{'go_id': 'GO:0005667', 'go_name': 'transcrip...","[{'location_id': 'SL-0086', 'location': 'Cytop...","[{'disease_id': 'umls:C0033860', 'disease_name...","[{'chembl_id': 'CHEMBL259571', 'drugbank_id': ...","[{'compound_name': nan, 'InChIKey': nan, 'SMIL...","[{'pubchem_assay_id': nan, 'assay_type': nan, ...","[{'stringdb_link_to': 'SCN4A', 'Ensembl': 'ENS..."
1,CHRNG,HGNC,ENSG00000196811,Ensembl,"[{'anatomical_entity_id': 'UBERON_0000178', 'a...","[{'pathway_id': nan, 'pathway_label': nan, 'pa...","[{'pathway_id': nan, 'pathway_label': nan, 'pa...",[{'pathway_label': 'Highly sodium permeable po...,"[{'go_id': 'GO:0015464', 'go_name': 'acetylcho...","[{'location_id': 'SL-0219', 'location': 'Posts...","[{'disease_id': 'umls:C0085631', 'disease_name...","[{'chembl_id': 'CHEMBL1200641', 'drugbank_id':...","[{'compound_name': nan, 'InChIKey': nan, 'SMIL...","[{'pubchem_assay_id': nan, 'assay_type': nan, ...","[{'stringdb_link_to': 'SCN4A', 'Ensembl': 'ENS..."
2,DMD,HGNC,ENSG00000198947,Ensembl,"[{'anatomical_entity_id': 'UBERON_0000178', 'a...","[{'pathway_id': nan, 'pathway_label': nan, 'pa...","[{'pathway_id': 'WP2858', 'pathway_label': 'Ec...",[{'pathway_label': 'Striated Muscle Contractio...,"[{'go_id': 'GO:0016010', 'go_name': 'dystrophi...","[{'location_id': 'SL-0039', 'location': 'Cell ...","[{'disease_id': 'umls:C0013264', 'disease_name...","[{'chembl_id': 'CHEMBL2108278', 'drugbank_id':...","[{'compound_name': nan, 'InChIKey': nan, 'SMIL...","[{'pubchem_assay_id': nan, 'assay_type': nan, ...",[]
3,HTR3A,HGNC,ENSG00000166736,Ensembl,"[{'anatomical_entity_id': 'UBERON_0000178', 'a...","[{'pathway_id': nan, 'pathway_label': nan, 'pa...","[{'pathway_id': 'WP706', 'pathway_label': 'Sud...",[{'pathway_label': 'Neurotransmitter receptors...,"[{'go_id': 'GO:1904602', 'go_name': 'serotonin...","[{'location_id': 'SL-0219', 'location': 'Posts...","[{'disease_id': 'EFO_0003843', 'disease_name':...","[{'chembl_id': 'CHEMBL56564', 'drugbank_id': '...","[{'compound_name': nan, 'InChIKey': nan, 'SMIL...","[{'pubchem_assay_id': nan, 'assay_type': nan, ...","[{'stringdb_link_to': 'DMD', 'Ensembl': 'ENSP0..."


In [32]:
combined_df.shape

(50, 15)

### Exporting the database in pickle format

In [33]:
with open("combined_df.pkl", "wb") as out:
    pickle.dump(combined_df, out)

## Creating a graph from the annotated dataframe

In [34]:
fuse_df = generator.load_dataframe_from_pickle("combined_df.pkl")

In [35]:
fuse_df.head()

,identifier,identifier.source,target,target.source,Bgee,MINERVA,WikiPathways,OpenTargets_Reactome,OpenTargets_GO,OpenTargets_Location,OpenTargets_Diseases,OpenTargets_Compounds,MolMeDB_transporter_inhibitor,PubChem_Assays,StringDB
0,AHR,HGNC,ENSG00000106546,Ensembl,"[{'anatomical_entity_id': 'UBERON_0000178', 'a...","[{'pathway_id': 953.0, 'pathway_label': 'Kynur...","[{'pathway_id': 'WP5044', 'pathway_label': 'Ky...","[{'pathway_label': 'Endogenous sterols', 'path...","[{'go_id': 'GO:0005667', 'go_name': 'transcrip...","[{'location_id': 'SL-0086', 'location': 'Cytop...","[{'disease_id': 'umls:C0033860', 'disease_name...","[{'chembl_id': 'CHEMBL259571', 'drugbank_id': ...","[{'compound_name': nan, 'InChIKey': nan, 'SMIL...","[{'pubchem_assay_id': nan, 'assay_type': nan, ...","[{'stringdb_link_to': 'SCN4A', 'Ensembl': 'ENS..."
1,CHRNG,HGNC,ENSG00000196811,Ensembl,"[{'anatomical_entity_id': 'UBERON_0000178', 'a...","[{'pathway_id': nan, 'pathway_label': nan, 'pa...","[{'pathway_id': nan, 'pathway_label': nan, 'pa...",[{'pathway_label': 'Highly sodium permeable po...,"[{'go_id': 'GO:0015464', 'go_name': 'acetylcho...","[{'location_id': 'SL-0219', 'location': 'Posts...","[{'disease_id': 'umls:C0085631', 'disease_name...","[{'chembl_id': 'CHEMBL1200641', 'drugbank_id':...","[{'compound_name': nan, 'InChIKey': nan, 'SMIL...","[{'pubchem_assay_id': nan, 'assay_type': nan, ...","[{'stringdb_link_to': 'SCN4A', 'Ensembl': 'ENS..."
2,DMD,HGNC,ENSG00000198947,Ensembl,"[{'anatomical_entity_id': 'UBERON_0000178', 'a...","[{'pathway_id': nan, 'pathway_label': nan, 'pa...","[{'pathway_id': 'WP2858', 'pathway_label': 'Ec...",[{'pathway_label': 'Striated Muscle Contractio...,"[{'go_id': 'GO:0016010', 'go_name': 'dystrophi...","[{'location_id': 'SL-0039', 'location': 'Cell ...","[{'disease_id': 'umls:C0013264', 'disease_name...","[{'chembl_id': 'CHEMBL2108278', 'drugbank_id':...","[{'compound_name': nan, 'InChIKey': nan, 'SMIL...","[{'pubchem_assay_id': nan, 'assay_type': nan, ...",[]
3,HTR3A,HGNC,ENSG00000166736,Ensembl,"[{'anatomical_entity_id': 'UBERON_0000178', 'a...","[{'pathway_id': nan, 'pathway_label': nan, 'pa...","[{'pathway_id': 'WP706', 'pathway_label': 'Sud...",[{'pathway_label': 'Neurotransmitter receptors...,"[{'go_id': 'GO:1904602', 'go_name': 'serotonin...","[{'location_id': 'SL-0219', 'location': 'Posts...","[{'disease_id': 'EFO_0003843', 'disease_name':...","[{'chembl_id': 'CHEMBL56564', 'drugbank_id': '...","[{'compound_name': nan, 'InChIKey': nan, 'SMIL...","[{'pubchem_assay_id': nan, 'assay_type': nan, ...","[{'stringdb_link_to': 'DMD', 'Ensembl': 'ENSP0..."
4,SCN4A,HGNC,ENSG00000007314,Ensembl,"[{'anatomical_entity_id': 'UBERON_0000178', 'a...","[{'pathway_id': nan, 'pathway_label': nan, 'pa...","[{'pathway_id': nan, 'pathway_label': nan, 'pa...",[{'pathway_label': 'Phase 0 - rapid depolarisa...,"[{'go_id': 'GO:0035725', 'go_name': 'sodium io...","[{'location_id': 'SL-0039', 'location': 'Cell ...","[{'disease_id': 'EFO_0000432', 'disease_name':...","[{'chembl_id': 'CHEMBL1077896', 'drugbank_id':...","[{'compound_name': '3-phenyl-1h-pyrazole', 'In...","[{'pubchem_assay_id': nan, 'assay_type': nan, ...",[]


In [36]:
pygraph = generator.networkx_graph(fuse_df)

### Store the graph

In [37]:
with open("networkx_graph.pkl", "wb") as out:
    pickle.dump(pygraph, out)

## Visualize the graph

In [38]:
# pos = nx.circular_layout(pygraph)

# plt.figure(3, figsize=(30, 30))
# nx.draw(pygraph, pos)
# plt.show()

In [ ]:
from pyBiodatafuse.graph import cytoscape, neo4j

neo4j.save_graph_to_graphml(pygraph, output_path="graph_to-test.graphml")
cytoscape.load_graph(pygraph, network_name="test")